<h1> GENRE: POP </h1>

<h3>(1) Access the data from the dataset </h3>


In [1]:
%%capture
%pip install datasets

from datasets import load_dataset

ds = load_dataset("sebastiandizon/genius-song-lyrics")

In [2]:
import heapq
from datasets import Dataset
type_of_music= 'misc'
TOP_K = 1000
decade_heaps = {70: [], 80: [], 90: [], 00: [], 10:[], 20:[]}  # z.B. 70=70er, 0=00er etc.

def get_decade(year):
    if 1970 <= year <= 1979:
        return 70
    elif 1980 <= year <= 1989:
        return 80
    elif 1990 <= year <= 1999:
        return 90
    elif 2000 <= year <= 2009:
        return 00
    elif 2010 <= year <= 2019:
        return 10
    elif 2020 <= year <= 2029:
        return 20
    return None

i = 0  # Zähler für tie-break
for song in ds["train"]:
    if song["tag"] != type_of_music:
        continue
    elif song["language"] != "en":
        continue
    dec = get_decade(song["year"])
    if dec is None:
        continue

    views = song["views"]
    heap = decade_heaps[dec]

    # Das Tupel enthält jetzt (views, i, song).
    # Der zweite Wert i ist rein für Tie-Break (und garantiert sortierbar).
    if len(heap) < TOP_K:
        heapq.heappush(heap, (views, i, song))
    else:
        # Vergleiche nur, ob aktuelle views (oder Tie-Break i) größer sind 
        # als das Minimum im Heap (heap[0]).
        if (views, i) > (heap[0][0], heap[0][1]):
            heapq.heapreplace(heap, (views, i, song))
    i += 1

# Am Ende hat jede Dekade max. 1000 Elemente.
decade_toplists = {}
for dec, heap in decade_heaps.items():
    # Sortieren nach (views, i) absteigend.
    # Falls dir der Index egal ist, kannst du einfach nur nach views sortieren.
    sorted_heap = sorted(heap, key=lambda x: (x[0], x[1]), reverse=True)
    # x[0] = views, x[1] = tie-break index, x[2] = song
    songs_only = [item[2] for item in sorted_heap]
    decade_toplists[dec] = songs_only

# Optional: zu HF Dataset konvertieren
subdataset_70 = Dataset.from_list(decade_toplists[70])
subdataset_80 = Dataset.from_list(decade_toplists[80])
subdataset_90 = Dataset.from_list(decade_toplists[90])
subdataset_00 = Dataset.from_list(decade_toplists[00])
subdataset_10 = Dataset.from_list(decade_toplists[10])
subdataset_20 = Dataset.from_list(decade_toplists[20])

path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/"
# Save to hard drive
subdataset_70.save_to_disk(path + "/Subdatasets/misc/subdataset_70_misc")
subdataset_80.save_to_disk(path + "/Subdatasets/misc/subdataset_80_misc")
subdataset_90.save_to_disk(path + "/Subdatasets/misc/subdataset_90_misc")
subdataset_00.save_to_disk(path + "/Subdatasets/misc/subdataset_00_misc")
subdataset_10.save_to_disk(path + "/Subdatasets/misc/subdataset_10_misc")
subdataset_20.save_to_disk(path + "/Subdatasets/misc/subdataset_20_misc")

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [4]:
path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/"
# Save to hard drive
subdataset_70.save_to_disk(path + "/Subdatasets/misc/subdataset_70_misc")
subdataset_80.save_to_disk(path + "/Subdatasets/misc/subdataset_80_misc")
subdataset_90.save_to_disk(path + "/Subdatasets/misc/subdataset_90_misc")
subdataset_00.save_to_disk(path + "/Subdatasets/misc/subdataset_00_misc")
subdataset_10.save_to_disk(path + "/Subdatasets/misc/subdataset_10_misc")
subdataset_20.save_to_disk(path + "/Subdatasets/misc/subdataset_20_misc")

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

<h2> LDA </h2>

<h2> Remove the bad words</h2>

In [1]:
subdataset_00[1]

NameError: name 'subdataset_00' is not defined

In [3]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("tushifire/ldnoobw")
path = kagglehub.dataset_download("sahib12/badwords")

#file = kagglehub.load_dataset("tushifire/ldnoobw",path)

print("Path to dataset files:", path)

swear_words_file = os.path.join(path, 'Terms-to-Block.csv')  # Replace 'en.txt' with the correct filename

with open(swear_words_file, 'r') as f:
    swear_words = set(f.read().splitlines())

print(f"!!!!Loaded {len(swear_words)} swear words.")

#print(swear_words)
print("Done")


Path to dataset files: /home/maurice/.cache/kagglehub/datasets/sahib12/badwords/versions/1
!!!!Loaded 2130 swear words.
Done


In [4]:
import gensim
import re
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from gensim.parsing.preprocessing import remove_stopwords

# Sicherstellen, dass NLTK-Daten heruntergeladen sind
nltk.download('stopwords')
nltk.download('punkt')

# Beispiel-Liste von Strings


# Stopwords definieren
stop_words = set(stopwords.words('english')+["like", "oh", "na", "la", "yo", "get", "yeah", "got","verse", "chorus", "hook", "bridge", "outro", "intro"])
print("Stopwords:", stop_words)
# Preprocessing: Tokenisieren, Stopwords entfernen, Kleinbuchstaben
def preprocess(doc):
    doc = re.sub(r"[^a-zA-Z0-9\s]", " ", doc) # delete unncessary things
    #doc = remove_stopwords(doc.lower())
    doc = remove_repeated_lines(doc)
    
    tokens = word_tokenize(doc.lower())  # Kleinbuchstaben und Tokenisierung
    
    tokens = [word for word in tokens if word not in stop_words]  # Stopwords entfernen
    
    return tokens

def remove_repeated_lines(doc):
    lines = doc.split("\n")
    return "\n".join(sorted(set(lines), key=lines.index))

def sort_out_bad_words(doc,word_list, replacement=" "):
    """
    Ersetzt Wörter in einem Text, wenn sie in einer Liste vorkommen.
    
    :param text: Der Eingabetext (String).
    :param word_list: Liste der Wörter, die ersetzt werden sollen.
    :param replacement: Der Text, durch den die Wörter ersetzt werden sollen.
    :return: Der bearbeitete Text.
    """
    # Erstelle ein Set für einen schnelleren Lookup
    word_set = set(word_list)
    
    # Splitte den Text in Wörter
    words = doc.split()
    
    # Ersetze jedes Wort, falls es in der Liste vorkommt
    replaced_words = [replacement if word.lower() in word_set else word for word in words]
    
    # Füge den Text wieder zusammen
    return " ".join(replaced_words)
 

def process_subdatasets(subdataset):
    # Get rid of the bad words and replace it with "swear word"
    without_swear = [sort_out_bad_words(doc, swear_words ) for doc in subdataset]

    # Preprocessing für alle Dokumente anwenden
    processed_docs = [preprocess(doc) for doc in without_swear]

    if stop_words in processed_docs:
        print("FEHLER: Stopwords nicht entfernt!")
        
    # Erstellen eines Wörterbuchs
    dictionary = Dictionary(processed_docs)

    # Erstellen eines Bag-of-Words-Korpus
    corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

    # LDA-Modell trainieren
    lda_model = gensim.models.LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=5,  # Anzahl der Themen
        random_state=42,
        passes=40,  # Anzahl der Durchgänge
        alpha='symmetric',
        eta='auto'
        #alpha='auto'  # Dirichlet-Hyperparameter Entscheidung zwischen Auto und symetric
    )
    
    id2word = dictionary
    dict_topic = {}
    # Themen anzeigen
    #print("Themen und ihre Schlüsselwörter:")
    for idx, topic in lda_model.print_topics(num_words=5):
        #print(f"Topic {idx}: {topic}")
        dict_topic[idx] = topic 

    return dict_topic


Stopwords: {"mightn't", 'ourselves', 'do', 'll', 'chorus', 'wasn', 'myself', 'for', 'was', 'most', 'each', 'nor', 'yo', 'can', 'ours', 'shouldn', 'himself', 'during', 'by', 'down', 'further', 'm', "isn't", "wouldn't", 'hook', 'on', 'am', 'again', 'mightn', "haven't", "you'd", "she's", 'does', 'such', "doesn't", 'there', 'be', 'now', 'and', 'from', 'whom', 'no', 'own', "hadn't", 'wouldn', 'needn', "weren't", 'hasn', 'like', 'more', 'just', "aren't", 'then', 'once', 'an', 'same', 'at', 'got', 'me', 'our', "couldn't", 'i', 'it', 'what', 'he', 'until', 'after', 'being', "should've", 'hadn', 'having', 'of', 'mustn', 'don', 'so', 'here', 'about', 'didn', 'over', 'get', 'or', 'is', 'above', 'in', 'yeah', 'few', 'off', "don't", 'theirs', 'had', 'any', "it's", 'na', 'you', 'y', 'him', 'other', 'which', 'ain', "mustn't", 'hers', 'those', 'with', 'if', 'yourselves', 'not', 'to', 've', 'yours', 'were', 'why', "hasn't", 'because', 'up', 'against', 'd', 'aren', 's', "didn't", 'isn', 'their', 'shan',

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maurice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/maurice/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
amount = 1000
from datasets import load_from_disk

path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Subdatasets/misc/"


rock_70_lyrics = process_subdatasets(load_from_disk(path + 'subdataset_70_misc')['lyrics'][:amount])
rock_80_lyrics = process_subdatasets(load_from_disk(path + "subdataset_80_misc")['lyrics'][:amount])
rock_90_lyrics = process_subdatasets(load_from_disk(path + "subdataset_90_misc")['lyrics'][:amount])
rock_00_lyrics = process_subdatasets(load_from_disk(path + "subdataset_00_misc")['lyrics'][:amount])
rock_10_lyrics = process_subdatasets(load_from_disk(path + "subdataset_10_misc")['lyrics'][:amount])
rock_20_lyrics = process_subdatasets(load_from_disk(path + "subdataset_20_misc")['lyrics'][:amount])

list_of_lyrics = {
    "70s": rock_70_lyrics,
    "80s": rock_80_lyrics,
    "90s": rock_90_lyrics,
    "00s": rock_00_lyrics,
    "10s": rock_10_lyrics,
    "20s": rock_20_lyrics
}

import csv

data_for_csv = []

for decade_name, decade in list_of_lyrics.items():
    i = 0
    for topics in decade:
        i = i + 1
        data_for_csv.append({
            "Method": "LDA",
            "Genre": "Rock",
            "Decade": decade_name,
            "Topic": str(decade[topics])
        })

path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/"
# Datei schreiben
output_file = path + "Analysis_Folder/LDA/topics_misc_LDA.csv"
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Method", "Genre", "Decade", "Topic"])
    writer.writeheader()  
    writer.writerows(data_for_csv) 

print(f"CSV-Datei '{output_file}' wurde erfolgreich erstellt!")


FileNotFoundError: [Errno 2] No such file or directory: '/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Analysis_Folder/LDA/topics_misc_LDA.csv'

In [6]:
path = "/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/"
# Datei schreiben
output_file = path + "Analysis_Folder/topics_misc_LDA.csv"
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Method", "Genre", "Decade", "Topic"])
    writer.writeheader()  
    writer.writerows(data_for_csv) 

print(f"CSV-Datei '{output_file}' wurde erfolgreich erstellt!")

CSV-Datei '/home/maurice/Dokumente/3rd_Semester_LiU/TDDE16_Projekt/TDDE16_Song_Lyrics_Analysis/Analysis_Folder/topics_misc_LDA.csv' wurde erfolgreich erstellt!


# tipps von ChaGPT
https://chatgpt.com/c/676f1ffc-b134-8012-9f03-6bd466bda148

- Herausfinden, wieviele Topics am geeingesten sind
- warum sind weiterhin so viele Stop words enthalten
- wieviele Lieder nehm ich überhaupt

In [ ]:
print(len(subdataset_80))

In [ ]:
print(len(subdataset_90))

<h1> BERT TOPIC MODELLING </h1>


In [69]:
import gensim
import re
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from gensim.parsing.preprocessing import remove_stopwords

# Sicherstellen, dass NLTK-Daten heruntergeladen sind
nltk.download('stopwords')
nltk.download('punkt')

# Beispiel-Liste von Strings


# Stopwords definieren
stop_words = set(stopwords.words('english')+["like", "oh", "na", "la", "yo", "get", "the","and", "of", "on", "[Intro]" ,"[Storyteller]" ])

# Preprocessing: Tokenisieren, Stopwords entfernen, Kleinbuchstaben
def preprocess(doc):
    list_of_tokens = []
    for i in doc:
        new_text = re.sub(r"[^a-zA-Z0-9\s]", " ", i) # delete unncessary things
        #doc = remove_stopwords(doc.lower())
        #doc = remove_repeated_lines(doc)
        
        tokens = word_tokenize(new_text.lower())  
        
        tokens = [word for word in tokens if word not in stop_words]
        list_of_tokens.append(tokens)
    
    return list_of_tokens

def remove_repeated_lines(doc):
    # doc ist eine Liste von Strings, z.B. ["Zeile1", "Zeile2", ...]
    # Wiederholte Zeilen entfernen und Reihenfolge beibehalten
    unique_lines = []
    seen = set()
    for line in doc:
        if line not in seen:
            unique_lines.append(line)
            seen.add(line)
    # Am Ende kannst du wieder eine Liste zurückgeben:
    return unique_lines

def sort_out_bad_words(doc,word_list, replacement="swear"):
    # Erstelle ein Set für einen schnelleren Lookup
    word_set = set(word_list)
    
    # Splitte den Text in Wörter
    words = doc.split()
    
    # Ersetze jedes Wort, falls es in der Liste vorkommt
    replaced_words = [replacement if word.lower() in word_set else word for word in words]
    
    # Füge den Text wieder zusammen
    return " ".join(replaced_words)

print("Done")


Done


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maurice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/maurice/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [70]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import csv

def processBertTopics(dataset,sentenceTransformer):
    # Initialize your topic model
    #embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
    
    embedding_model = SentenceTransformer(sentenceTransformer)
    topic_model = BERTopic(embedding_model=embedding_model)
    # Fit and transform the text data
    without_swear = remove_repeated_lines(dataset)
    without_swear = preprocess(without_swear)
    
    without_swear = [sort_out_bad_words(doc, swear_words ) for doc in dataset]
    print(type(without_swear))
    print(without_swear[1])
    print(without_swear[2])
    print(without_swear[3])

    topics, probs = topic_model.fit_transform(without_swear)

    str_topic_with_probs = []
    #for t, p in zip(topics, probs):
    #    str_topic_with_probs.append(str(t) + ": " + str(p))
        
    #return str_topic_with_probs
    #print(topic_model.get_topic_info())
    print(type(topic_model.get_topics()))
    print(topic_model.get_topics())
    return topic_model.get_topics()
    
# Print or analyze the results
#print("Topics:", topics)
#print("Probabilities:", probs)
print("Done")


Done


In [16]:
import torch
from transformers import AutoModel, AutoTokenizer

# Modell und Tokenizer laden
bertweet = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

# Anzahl der Songtexte, die verarbeitet werden sollen
amount = 10  # Beispiel: Anzahl der Texte, die verarbeitet werden sollen

# INPUT TWEET IST BEREITS NORMALISIERT!
# Tokenisierung und Padding/Truncation
lyrics = subdataset_90['lyrics'][:amount]
inputs = tokenizer(lyrics, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Verarbeitung der Eingabe durch das Modell
with torch.no_grad():
    features = bertweet(**inputs)  # Models outputs sind jetzt Tupel

# Ausgabe der Features (optional: erster Layer oder hidden states)
print(features.last_hidden_state)  # Beispielausgabe
print("Done")


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


IndexError: index out of range in self

In [71]:
import time
from datasets import load_from_disk
def run_analytics_bert_different_Sentence_Transformers(amount_per_iteration,name_of_run, sentence_transformer):

    start_time = time.time()
    amount = amount_per_iteration

    #rap_70_lyrics_bert = processBertTopics(subdataset_70['lyrics'][:amount],sentence_transformer)
    #rap_80_lyrics_bert = processBertTopics(subdataset_80['lyrics'][:amount],sentence_transformer)
    #rap_90_lyrics_bert = processBertTopics(subdataset_90['lyrics'][:amount],sentence_transformer)
    #rap_00_lyrics_bert = processBertTopics(subdataset_00['lyrics'][:amount],sentence_transformer)
    #rap_10_lyrics_bert = processBertTopics(subdataset_10['lyrics'][:amount],sentence_transformer)
    #rap_20_lyrics_bert = processBertTopics(subdataset_20['lyrics'][:amount],sentence_transformer)

    
    rap_70_lyrics_bert = processBertTopics(load_from_disk("subdataset_70_rap")['lyrics'][:amount],sentence_transformer)
    rap_80_lyrics_bert = processBertTopics(load_from_disk("subdataset_80_rap")['lyrics'][:amount],sentence_transformer)
    rap_90_lyrics_bert = processBertTopics(load_from_disk("subdataset_90_rap")['lyrics'][:amount],sentence_transformer)
    rap_00_lyrics_bert = processBertTopics(load_from_disk("subdataset_00_rap")['lyrics'][:amount],sentence_transformer)
    rap_10_lyrics_bert = processBertTopics(load_from_disk("subdataset_10_rap")['lyrics'][:amount],sentence_transformer)
    rap_20_lyrics_bert = processBertTopics(load_from_disk("subdataset_20_rap")['lyrics'][:amount],sentence_transformer)
    
    

    list_of_lyrics_bert = {
        "70s": rap_70_lyrics_bert,
        "80s": rap_80_lyrics_bert,
        "90s": rap_90_lyrics_bert,
        "00s": rap_00_lyrics_bert,
        "10s": rap_10_lyrics_bert,
        "20s": rap_20_lyrics_bert,
    }
    end_time = time.time()

    amount_of_time = end_time - start_time
    
    data_for_csv_bert = []
    
    for decade_name, decade in list_of_lyrics_bert.items():
        i = 0
        
        for topics in decade:
            i = i + 1
            print(decade)
            print(type(decade))
            data_for_csv_bert.append({
                "Method": "BERT",
                "Genre": "Rap",
                "Decade": decade_name,
                "Topic": str(decade[topics])
            })
    data_for_csv_bert.append({
                "Method": "TIME",
                "Genre": "TIME",
                "Decade": "TIME",
                "Topic": str(amount_of_time)
            })
    
    # Datei schreiben
    name_of_file = "topics_rap_BERT_" + str(sentence_transformer) + str(name_of_run) + ".csv"
    output_file = os.path.join("Analysis_Folder/BERT/", name_of_file)
    #output_file = "Analysis_Folder/BERT/topics_rap_BERT_.csv"
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["Method", "Genre", "Decade", "Topic"])
        writer.writeheader()  # Schreibe die Kopfzeile
        writer.writerows(data_for_csv_bert)  # Schreibe die Daten
    
    print(f"CSV-Datei '{output_file}' wurde erfolgreich erstellt!")
    print("Done")


In [72]:
run_analytics_bert_different_Sentence_Transformers(10,"TEST_MIST","paraphrase-Mpnet-base-v2")

{'vag1na', 'boned', 'beastiality', 'a_s_s', 'tinkle', 'masterbaiter', '"niggers,"', 'bimbos', 'wigger', 'opium', '"schlong,"', 'fitt*', 'cunt', 'screwed', 'nutsack', 'herp', 'bastards', 'boners', 'prick', 'clusterfuck', '"frigg,"', 'douche', '"goddamn,"', '"x-rated,"', '"dildo,"', 'mothafucka', 'ma5terb8', 'hiv', 'loin', '"dickweed,"', '"shithead,"', '"spik,"', '"bone,"', '"homoey,"', '"bollok,"', '"hemp,"', 'ballbag', 'hoorem whore', 'clitty', 'fcuker', '"cocaine,"', '"bootee,"', 'doggin', '"penetrate,"', 'w0p', 'boobies', 'pigfucker', 'fingerfuck ', 'dickhead', 'tittyfucker', 'dick-ish', '"crack,"', 'polac', 'fucking', 'fack', 'spunk', 'fvck', 'masterbating', 'sharmute', '"erect,"', '"maxi,"', '"fart,"', 'cum freak', '"prostitute,"', 'bestial', 'wank', 'shaggin', 'Fuken', 'fukwit', 'fucknugget', 'shagger', '"orally,"', '"sex,"', 'whored', 'lezbo', '"fisting,"', 'lezbos', 'qweir', '"erection,"', '"dickwhipper,"', '"mutherfucking,"', 'cyberfuckers', '"lech,"', 'shag', 'babes', 'erotic'

In [87]:
topic_model.get_topics(0)

NameError: name 'topic_model' is not defined